In [42]:
import pandas as pd
import numpy as np
import jutil

In [21]:
df = pd.read_csv('000-mark-0101.csv',usecols=range(1,11),dtype={'college_code':np.object,'course_code':np.object,'speciality_code':np.object,'student_id':np.object})
df.head()

,college_code,course_code,course_name,course_type,mark,pmark,speciality_code,speciality_name,student_id,term_order
0,01,6111120,大学物理实验Ⅱ,必,5,73.0,0101,土木工程,19950854,-2
1,01,9010110,体育Ⅰ,必,7,80.0,0101,土木工程,19950854,-3
2,01,6111120,大学物理实验Ⅱ,必,1,61.0,0101,土木工程,19960006,2
3,01,9010120,体育Ⅱ,必,6,76.0,0101,土木工程,19960006,2
4,01,7035000,马克思主义哲学原理,必,8,83.0,0101,土木工程,19960037,4


In [22]:
df.dtypes

college_code        object
course_code         object
course_name         object
course_type         object
mark                 int64
pmark              float64
speciality_code     object
speciality_name     object
student_id          object
term_order           int64
dtype: object

In [23]:
data = df[~df.course_name.str.startswith('形势与政策')] # 去除行政课

In [26]:
data.count()

college_code       620013
course_code        620013
course_name        620013
course_type        620013
mark               620013
pmark              620013
speciality_code    620013
speciality_name    620013
student_id         620013
term_order         620013
dtype: int64

In [25]:
dg = data.groupby('student_id').groups

input_list = []
iii = None
try:
    for k in dg:
        ids = dg[k]
        iii = ids
        items = [ data.iloc[id].get('course_code')+'_'+str(data.iloc[id].get('mark')) for id in ids]
        items = list(set(items)) # 去重
        input_list.append((k,items))
except:
    print(iii)
len(input_list)

Int64Index([246337, 246338, 246339, 246340, 246341, 246342, 246343, 246345,
            246346, 257806, 257807, 257808, 257809, 257813, 257814, 257815,
            257816, 538126, 538127, 538128, 538871, 538872, 538873, 538874,
            538875, 538876, 538877, 538878, 538879, 538880, 538881, 538882,
            593040, 593041, 593042, 593043, 593044, 593045, 593046, 593047,
            593048, 593049, 593050, 600854, 600855, 600856, 600857, 600858,
            600859, 600860, 600862, 600863, 604156, 604157, 604158, 604159,
            604160, 604161, 604162, 604163, 604164, 616228, 616229, 616230,
            616231, 616232, 616233, 616234, 616235, 616236, 616237, 616238,
            616239, 616242, 616243, 620011, 620012, 620013, 620014, 620015,
            620016, 620017, 620018, 620019, 620020],
           dtype='int64')


9386

In [54]:
from pyspark.ml.fpm import FPGrowth

ddd = spark.createDataFrame(input_list,['id','items'])

fpG = FPGrowth(itemsCol="items",minSupport=0.1,minConfidence=0.2)
model = fpG.fit(ddd)

In [55]:
model.freqItemsets.show(n=10,truncate=False)

+-----------+----+
|items      |freq|
+-----------+----+
|[9010110_8]|1083|
|[9990004_8]|1020|
|[0132410_1]|1241|
|[9990004_9]|1662|
|[9990226_9]|1378|
|[8010230_1]|2176|
|[0153570_1]|1177|
|[8010220_5]|956 |
|[6111120_4]|1316|
|[9010130_9]|1853|
+-----------+----+
only showing top 10 rows



In [56]:
model.associationRules.show(truncate=False)

+-----------+-----------+-------------------+
|antecedent |consequent |confidence         |
+-----------+-----------+-------------------+
|[6335110_0]|[6321920_0]|0.4203770557561171 |
|[6322010_0]|[6321920_0]|0.4025679758308157 |
|[6322020_0]|[6321920_0]|0.41833466773418737|
|[6321920_0]|[6322020_0]|0.3254437869822485 |
|[6321920_0]|[6335110_0]|0.3263780753659296 |
|[6321920_0]|[6322010_0]|0.3319838056680162 |
|[6321920_0]|[6321910_0]|0.3453752725007786 |
|[6321920_0]|[6322010_1]|0.30177514792899407|
|[6321920_0]|[6011320_0]|0.29554655870445345|
|[6321920_0]|[6321920_1]|0.31236374961071317|
|[6322010_1]|[6321920_0]|0.39214892755969244|
|[6011320_0]|[6321920_0]|0.41568112133158125|
|[6321910_0]|[6321920_0]|0.41944024205748864|
|[6321920_1]|[6321920_0]|0.39581689029202843|
+-----------+-----------+-------------------+



In [ ]:
pdc = model.associationRules.toPandas()